**Getting data for below scenario:**

- Flights from 10th Oct (departure) - 5th Jan
- Adult 1 Passenger only
- Economy
- One way

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from selenium import webdriver
from selenium.webdriver import Chrome

In [3]:
driver = Chrome()
driver.get('https://www.kayak.ie/flights/DUB-LHR/2024-12-25?sort=bestflight_a')

In [4]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
'''
flight_details = []
for item in soup('div'):
    try:
        # print(item['class'][0])
        if item['class'][0] == "J0g6-operator-text":
            flight_details.append([item.contents[0]])
    except:
        pass

print(flight_details)
#<span class="JWEO-stops-text JWEO-mod-warning">2 stops</span>
for item in soup('span'):
    try:
        # print(item['class'][0])
        if item['class'][0] == "JWEO-stops-text":
            flight_details.append([item.contents[0]])
    except:
        pass

print(flight_details)
'''

In [6]:
# print(soup)
# <div class="J0g6-operator-text">British Airways</div>

# flight container:class = yuAt yuAt-pres-rounded yuAt-mod-box-shadow

# price element <div class="f8F1-price-text">€333</div>

# duration element: <div class="vmXl vmXl-mod-variant-default">23h 40m</div>

#<div class="vmXl vmXl-mod-variant-large"><span>14:35</span><span class="aOlM"> – </span><span>14:15<sup class="VY2U-adendum" title="Flight lands the next day">+1</sup></span></div>

flight_details = []
flight_containers = soup.findAll('div', class_="yuAt yuAt-pres-rounded yuAt-mod-box-shadow")
for item in flight_containers:
    try:
        flight_name = item.find('div', class_="J0g6-operator-text").get_text(strip = True)

        stops = item.find('span', class_="JWEO-stops-text").get_text(strip = True)

        price = item.find('div', class_="f8F1-price-text").get_text(strip = True)

        #Extracting duration by skipping time and stops
        duration = None
        divs = item.findAll('div', class_="vmXl vmXl-mod-variant-default")

        for div in divs:
            if not div.find('span'):  # If there are no spans inside the div, it's likely the duration
                duration = div.get_text(strip=True)
                break  # Found the duration, no need to search further

         # Extracting departure and arrival times
        times_div = item.find('div', class_="vmXl vmXl-mod-variant-large")
        if times_div:
            times = times_div.get_text(strip=True).split("–")
            departure_time = times[0].strip() if len(times) > 0 else "Departure not found"
            arrival_time = times[1].strip() if len(times) > 1 else "Arrival not found"
        else:
            departure_time, arrival_time = "Departure not found", "Arrival not found"

        flight_details.append([flight_name, stops, price, duration, departure_time, arrival_time])
        
    except:
        pass




## Issues
1. Multiple elements sharing same class making it hard to extract relevant info
    eg.  class="vmXl vmXl-mod-variant-default" is being shared with start time of flight, duration and number of stops.

# Comments #
- +1 in flight arrival time indicates that the flight lands on the next day.
